In [ ]:
import re

val = "varchar"
z = re.match(r"(\w+)(?=\(([0-9]{1,4})\))?",val)
print(z.groups())

val = "decimal(20,4)"
z = re.match(r"(\w+)(?=\(([0-9]{1,2},[0-9]{1,2})\))",val)
print(z.groups())

In [ ]:
import sys

MODULE_FULL_PATH = 'd:/GitHUB_Sandbox'

sys.path.insert(1, MODULE_FULL_PATH)


In [ ]:
import findspark
findspark.init()

from pyspark.sql import types
from sandbox.spark_etl.common.data_validator.dtype_mapper_hive import dtypes_mapper_hive

In [ ]:
dict_translation = dtypes_mapper_hive()
print(dict_translation.translate_dtype_to_sparkdtype(raw_dtype="varchar(10)"))

In [ ]:
import findspark
findspark.init()
from json import loads
from sandbox.spark_etl.common.data_validator.source_schema import source_schema
from sandbox.spark_etl.common.data_validator.dtype_mapper_hive import dtypes_mapper_hive


def format_output_to_list_of_dict(output_json : dict) -> dict:
    
    for field_md in output_json["fields"]:
        print(field_md)
        
    # {"fields":[{"metadata":{"datatype_precision":"10","datatype_scale":null,"format":"","source_datatype":"varchar(10)","spark_datatype":"string"},"name":"COLUMN_1","nullable":false,"type":"string"},{"metadata":{"datatype_precision":"10","datatype_scale":"4","format":"","source_datatype":"decimal(10,4)","spark_datatype":"decimal(10,4)"},"name":"COLUMN_2","nullable":true,"type":"decimal(10,4)"},{"metadata":{"datatype_precision":null,"datatype_scale":null,"format":"","source_datatype":"timestamp","spark_datatype":"timestamp"},"name":"COLUMN_2","nullable":true,"type":"timestamp"}],"type":"struct"}

raw_schema_MD = [
            {
            "column_name" : "COLUMN_1",
            "required" : True,
            "data_type" : "varchar(10)",
            "format" : ""
            }
            ,
            {
                "column_name" : "COLUMN_2",
                "required" : False,
                "data_type" : "decimal(10,4)",
                "format" : ""
            },
            {
                "column_name" : "COLUMN_2",
                "required" : False,
                "data_type" : "timestamp",
                "format" : ""
            }        
        ]
# {"fields":[{"metadata":{"datatype_precision":"10","datatype_scale":null,"format":"","source_datatype":"varchar(10)","spark_datatype":"string"},"name":"COLUMN_1","nullable":false,"type":"string"}
#            ,{"metadata":{"datatype_precision":"10","datatype_scale":"4","format":"","source_datatype":"decimal(10,4)","spark_datatype":"decimal(10,4)"},"name":"COLUMN_2","nullable":true,"type":"decimal(10,4)"}]
#  ,"type":"struct"}
schema_extended = source_schema(list_of_dict=raw_schema_MD,dtype_translator=dtypes_mapper_hive()).get_struct_schema_with_sparkdtypes()
# print("HEY",schema_extended.json()  )
print("HEY",format_output_to_list_of_dict ( loads(schema_extended.json() ) ) )

result = [ {'metadata': {'datatype_precision': '10', 'datatype_scale': None, 'format': '', 'source_datatype': 'varchar(10)', 'spark_datatype': 'string'}, 'name': 'COLUMN_1', 'nullable': False, 'type': 'string'},
{'metadata': {'datatype_precision': '10', 'datatype_scale': '4', 'format': '', 'source_datatype': 'decimal(10,4)', 'spark_datatype': 'decimal(10,4)'}, 'name': 'COLUMN_2', 'nullable': True, 'type': 'decimal(10,4)'},
{'metadata': {'datatype_precision': None, 'datatype_scale': None, 'format': '', 'source_datatype': 'timestamp', 'spark_datatype': 'timestamp'}, 'name': 'COLUMN_2', 'nullable': True, 'type': 'timestamp'}
]

pairs = zip (result,loads(schema_extended.json() ) )
            
any( expected_val !=  result_val for expected_val,result_val in pairs )